# Feature ennginering
- adding alternative rankings
- using only historical data

Opens file

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

db = pd.read_csv('db/out/wta_std.csv')

db['date'] = pd.to_datetime(db['date'], format='%Y-%m-%d')

db.dropna()

ELO and GLICKO rankings


In [ ]:
from utils.rating_elo import update_elo


players = pd.concat([db['name_P1'], db['name_P2']]).unique()
ratings = {player: 1500 for player in players}


db['elo_P1'] = np.nan
db['elo_P2'] = np.nan


for id, match in db.iterrows():
    player1 = match['name_P1']
    player2 = match['name_P2']
    player1_rating = ratings[player1]
    player2_rating = ratings[player2]
    db.at[id, 'elo_P1'] = player1_rating
    db.at[id, 'elo_P2'] = player2_rating
    ratings[player1], ratings[player2] = update_elo(
        player1_rating, player1_rating, match['y'])

In [ ]:
from utils.rating_glicko import update_glicko

DEFAULT_RD = 350


ratings_glico = {player: 1500 for player in players}

RDs = {player: DEFAULT_RD for player in players}

volatility = {player: 0.06 for player in players}


db['glicko_P1'] = np.nan
db['glicko_P2'] = np.nan
db['RD_P1'] = np.nan
db['RD_P2'] = np.nan


for id, row in db.iterrows():
    p1 = row['name_P1']
    p2 = row['name_P2']

    # Get current ratings and RDs
    p1_current_rating = ratings_glico[p1]
    p2_current_rating = ratings_glico[p2]
    p1_current_RD = RDs[p1]
    p2_current_RD = RDs[p2]
    p1_current_volatility = volatility[p1]
    p2_current_volatility = volatility[p2]

    # Store current ratings and RDs
    db.loc[id, 'glicko_P1'] = p1_current_rating
    db.loc[id, 'glicko_P2'] = p2_current_rating
    db.loc[id, 'RD_P1'] = p1_current_RD
    db.loc[id, 'RD_P2'] = p2_current_RD

    # Update ratings and RDs based on match result (P1 is assumed to be the winner)
    p1_new_rating, p1_new_RD = update_glicko(
        p1_current_rating, p1_current_RD, p2_current_rating, p2_current_RD, 1)
    p2_new_rating, p2_new_RD, = update_glicko(
        p2_current_rating, p2_current_RD, p1_current_rating, p1_current_RD, 0)

    # Update the ratings and RDs dictionary
    ratings_glico[p1] = p1_new_rating
    RDs[p1] = p1_new_RD
    ratings_glico[p2] = p2_new_rating
    RDs[p2] = p2_new_RD

Basic stats 

In [ ]:
from utils.features import basicfeatureCreator
basic_stats = basicfeatureCreator(db)

Player vs. Player stats

In [ ]:
from utils.vsFeatures import vsfeatureCreator

vsstats = vsfeatureCreator(db)

mean_pvp = pd.merge(
    basic_stats, vsstats, on=['match_id'], how='inner', suffixes=('', '_VS'))

mean_pvp.drop(
    ['draw_size', 'tourney_level', 'surface', 'hand_P1', 'hand_P2'], inplace=True, axis=1)

db = mean_pvp

testp1 = db.filter(regex='_P1_VS$', axis=1)

testp2 = db.filter(regex='_P2_VS$', axis=1)


feats = list(zip(testp1.columns, testp2.columns))


for col_a, col_b in feats:
    db[col_a[:-6] + '_vs'] = db[col_a] - db[col_b]

db.drop(testp1.columns, axis=1, inplace=True)

db.drop(testp2.columns, axis=1, inplace=True)

non_numeric_columns = db.select_dtypes(['object']).columns

db = db.drop(non_numeric_columns, axis=1)

testp1 = db.filter(regex='_P1$', axis=1)

testp2 = db.filter(regex='_P2$', axis=1)

feats = list(zip(testp1.columns, testp2.columns))

for col_a, col_b in feats:
    db[col_a[:-3]] = db[col_a] - db[col_b]

db.drop(testp1.columns, axis=1, inplace=True)

db.drop(testp2.columns, axis=1, inplace=True)

db.info()

db.to_csv(
    '../db/out/wta_s.csv', index=False)